In [1]:
pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

fastf1.Cache.enable_cache("f1_cache")
session_2024=fastf1.get_session(2024, 3, "R")
session_2024.load()
laps_2024=session_2024.laps[["Driver", "LapTime"]].copy()
laps_2024.dropna(subset=["LapTime"], inplace=True)
laps_2024["LapTime (s)"]=laps_2024["LapTime"].dt.total_seconds()

qualifying_2025=pd.DataFrame({
    "Driver": ["Lando Norris", "Oscar Piastri", "Max Verstappen", "George Russell", "Yuki Tsunoda", "Alexander Albon", "Charles Leclarc","Lewis Hamilton", "Pierre Gasly", "Carlos Sainz", "Lance Stroll", "Fernand Alonso"],
    
    "QualifyingTime (s)": [75.096, 75.180, 74.481, 75.546, 75.670, 75.737, 75.755, 75.973, 75.900, 76.062, 76.400, 76.500]
})
driver_mapping={
    "Lando Norris": "NOR", "Oscar Piastri": "PIA", "Max Verstappen": "VER", "George Russell": "RUS", "Yuki Tsunoda": "TSU", 
    "Alexander Albon": "ALB", "Charles Leclarc": "LEC", "Lewis Hamilton": "HAM", "Pierre Gasly": "GAS", "Carlos Sainz": "SAI", "Lance Stroll": "STR", "Fernand Alonso": "ALO"
}
qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)
merged_data=qualifying_2025.merge(laps_2024, left_on="DriverCode", right_on="Driver")
x=merged_data[["QualifyingTime (s)"]]
y=merged_data["LapTime (s)"]

if x.shape[0]==0:
    raise ValueError("Empty Dataset")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=39)
model= GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=39)
model.fit(x_train, y_train)
predicted_lap_times=model.predict(qualifying_2025[["QualifyingTime (s)"]])
qualifying_2025["PredictedRaceTime (s)"]=predicted_lap_times
qualifying_2025=qualifying_2025.sort_values(by="PredictedRaceTime (s)")
print("\n Predicted 2025 Australian GP Winner: \n")
print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])
y_pred=model.predict(x_test)
print(f"\n Model Error (MAE): {mean_absolute_error(y_test, y_pred):.2f} seconds")

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['55', '16', '4', '81', '11', '18', '22', '14', '27', '20', '23', '3', '10', '77', '24', '31', '63', '44', '1']



 Predicted 2025 Australian GP Winner: 

             Driver  PredictedRaceTime (s)
0      Lando Norris              82.711074
6   Charles Leclarc              83.079100
9      Carlos Sainz              83.618787
3    George Russell              83.887261
1     Oscar Piastri              84.329848
4      Yuki Tsunoda              84.418938
5   Alexander Albon              84.644237
10     Lance Stroll              84.691932
11   Fernand Alonso              85.114474
2    Max Verstappen              85.267834
8      Pierre Gasly              85.540019
7    Lewis Hamilton              86.093940

 Model Error (MAE): 3.37 seconds
